In [1]:
import numpy as np
import queue
import json
import ctypes
import os
import time
import pickle as pkl
import multiprocessing

import neuron
from neuron import nrn_dll_sym, h
from neuron.units import ms, mV
h.load_file("stdrun.hoc")
h.load_file("stdlib.hoc")
h.load_file("import3d.hoc")

import sys
sys.path.insert(1, "../../utils/")
import Stimuli, Pyr

In [2]:
segment_arrayID = 'ID96'
n = 5
data_dir = '../../data/morpho/'
duration = 100

## generate stimuli

In [3]:
print('generating stimuli')
stim_locs_file = [file for file in os.listdir(f'{data_dir}segment_arrays/') if segment_arrayID+'.npy' in file][0]
print(stim_locs_file)

stim_params = Stimuli.ExperimentalStimParams()
stim_scaffold = stim_params.stim_scaffold['pyr']
stim_locs = np.load(f'{data_dir}segment_arrays/{stim_locs_file}')

stimuli = Stimuli.MorphoStimuli('stimuli',stim_scaffold['stim_type_array'],stim_locs, stim_scaffold, duration)

num_events = sum([len(stimulus.event_times) for stimulus in stimuli.stimuli])
print(f'total number of events: {num_events}')

try: os.mkdir(f'{data_dir}nInputs_stimuli') 
except: pass
with open(f'{data_dir}nInputs_stimuli/morpho_stimuli_{segment_arrayID}_{n}.pkl', 'wb') as handle:
    pkl.dump(stimuli, handle, protocol=pkl.HIGHEST_PROTOCOL)

generating stimuli
segment_array_12.6Hz_ID96.npy
total number of events: 63


## load histories

In [4]:
print('loading histories')
histories_files = [file for file in os.listdir(f'{data_dir}random_histories/') if segment_arrayID+'.json' in file]
histories = []
for history_file in histories_files:
    with open(f'{data_dir}random_histories/{history_file}') as fin:
        histories.append(json.load(fin))
print(f'histories: {len(histories)}')
print()

loading histories
histories: 215



## simulate cell

In [5]:
'''start_time = time.time()
pyr = Pyr.Pyr()

pyr.add_stimuli(stimuli)

h.CVode().active(True)
h.celsius = 35
h.finitialize()
h.continuerun(duration)
print(f'simulation took {round(time.time() - start_time, 2)} sec to run')'''

"start_time = time.time()\npyr = Pyr.Pyr()\n\npyr.add_stimuli(stimuli)\n\nh.CVode().active(True)\nh.celsius = 35\nh.finitialize()\nh.continuerun(duration)\nprint(f'simulation took {round(time.time() - start_time, 2)} sec to run')"

In [6]:
# clear out all of NEURON
h("forall delete_section()")

1

## nInputs

In [7]:
def run_event_sim(stimuli, duration):
    pyr = Pyr.Pyr()
    #pyr.initialize_state_vars(history)
    pyr.add_stimuli(stimuli)
    
    h.CVode().active(True)
    h.celsius = 35
    h.finitialize()
    h.continuerun(duration)
    return list(pyr.spike_times)

In [8]:
def pyonevent(stim_times):
    start_time = time.time()
    stim_times = neuron.numpy_from_pointer(stim_times, 50*30)
    stim_times = stim_times.reshape([30,50])[:,:n]
    stim_times = stim_times - np.min(stim_times[stim_times > 0])
    
    # change the stimuli
    for i in range(30):
        stimuli.stimuli[i].event_times = list(stim_times[i, :])
    
    # pool is used to parallelize
    spikes = pool.starmap(
        run_event_sim,
        [
            [stimuli, (np.nanmax(stim_times)+50)]
        ]
    )[0]
    
    print(f'simulation time: {np.nanmax(stim_times) + 50}')
    print(f'real time: {time.time() - start_time}')
    print()
    return spikes

In [9]:
run_event_sim(stimuli, 50)

[17.90388721292464]

In [ ]:
# Set up communications between NEURON and python
double_ptr = ctypes.POINTER(ctypes.c_double)
on_event_proto = ctypes.CFUNCTYPE(ctypes.c_double, double_ptr)
# CFUNCTYPE parameters are as follows: (the return type, the input types which are 1) estims, 2) istims, 3) spikes

on_event_c = on_event_proto(pyonevent)
on_event_c_ptr = ctypes.cast(on_event_c, double_ptr)

nrn_hocobj_ptr = nrn_dll_sym("nrn_hocobj_ptr")
nrn_hocobj_ptr.restype = ctypes.py_object
event_callback_ptr = nrn_hocobj_ptr(ctypes.cast(on_event_c_ptr, double_ptr))

In [ ]:
pool = multiprocessing.Pool(1)

In [ ]:
ebneuron = h.ebneuron()
ebneuron._ref_on_event = event_callback_ptr
ebneuron_spikes = h.Vector()
ebneuron_nc = h.NetCon(ebneuron, None)
ebneuron_nc.record(ebneuron_spikes)

In [ ]:
netstims = []
netcons = []

for w, stimulus in enumerate(stimuli.stimuli):
    for stim_time in stimulus.event_times:
        netstims.append(h.NetStim())
        netstims[-1].start = stim_time * ms
        netstims[-1].number = 1
        netcons.append(h.NetCon(netstims[-1], ebneuron))
        netcons[-1].weight[0] = w +1 # weight (w) is used to indicate the unique location of the stimulus
        netcons[-1].delay = 0

In [ ]:
h.finitialize()
h.continuerun(duration)

In [ ]:
ebneuron.print()

In [ ]:
pool.close()